In [1]:
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras import Input, Model
from keras import backend as K
from keras.callbacks import TensorBoard
from keras.layers import Dense, LeakyReLU, BatchNormalization, ReLU, Reshape, UpSampling2D, Conv2D, Activation, concatenate, Flatten, Lambda, Concatenate, Embedding, GRU
from tensorflow.keras.optimizers import Adam

import matplotlib.pyplot as plt
import base64
import os
import numpy as np
import pandas as pd
from urllib import request
from PIL import Image
from IPython import display
import time
import imageio
from tensorflow_docs.vis import embed

In [2]:
datapath = 'dataset/full_emoji.csv'
df = pd.read_csv(datapath)

In [3]:
def toimg(img_url):
    request.urlretrieve(img_url, 'img')
    img = Image.open('img')
    return img.convert("RGB").resize((64, 64), Image.BILINEAR)

def imgtoarray(img_url):
    try:
        img = toimg(img_url)
        return np.asarray(img, dtype=np.float32)
    except TypeError:
        return None

In [4]:
images = []
for service in ['Apple', 'Google', 'Facebook', 'Windows']:
    for url in df[service].values:
        images.append(imgtoarray(url))
labels = df['name'].to_list()*4
len(images), len(labels)

/opt/homebrew/Caskroom/miniforge/base/envs/cp1/lib/python3.8/site-packages/PIL/Image.py:945: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


(7264, 7264)

In [5]:
df_new = pd.DataFrame(zip(images, labels), columns=['Image', 'Label'])
df_new.dropna(axis=0, inplace=True)
df_new.reset_index(drop=True, inplace=True)
df_new

,Image,Label
0,"[[[71.0, 112.0, 76.0], [71.0, 112.0, 76.0], [7...",grinning face
1,"[[[71.0, 112.0, 76.0], [71.0, 112.0, 76.0], [7...",grinning face with big eyes
2,"[[[71.0, 112.0, 76.0], [71.0, 112.0, 76.0], [7...",grinning face with smiling eyes
3,"[[[71.0, 112.0, 76.0], [71.0, 112.0, 76.0], [7...",beaming face with smiling eyes
4,"[[[71.0, 112.0, 76.0], [71.0, 112.0, 76.0], [7...",grinning squinting face
...,...,...
6808,"[[[71.0, 112.0, 76.0], [71.0, 112.0, 76.0], [7...",crossed flags
6809,"[[[71.0, 112.0, 76.0], [71.0, 112.0, 76.0], [7...",black flag
6810,"[[[71.0, 112.0, 76.0], [71.0, 112.0, 76.0], [7...",white flag
6811,"[[[71.0, 112.0, 76.0], [71.0, 112.0, 76.0], [7...",rainbow flag


In [6]:
import re
df_new['Label'] = df_new['Label'].apply(lambda x: ''.join(re.findall(r'[ A-Za-z0-9]', x))).values.tolist()
texts = df_new['Label'].values

tokenizer = Tokenizer(num_words=2000)
tokenizer.fit_on_texts(texts)
word_dic = tokenizer.word_index
vocab_size = len(word_dic)
vocab_size

1683

In [7]:
sequences = tokenizer.texts_to_sequences(texts)
padded = pad_sequences(sequences)
Images = np.stack(df_new['Image'].values)
Labels = np.asarray(padded, dtype=np.float32)

Images.shape, Labels.shape

((6813, 64, 64, 3), (6813, 6))

In [8]:
ths_hold = int(Images.shape[0]*0.8)
train_images = Images[:ths_hold, :,:,:]
test_images = Images[ths_hold:, :,:,:]
train_labels = Labels[:ths_hold, :]
test_labels = Labels[ths_hold:, :]

train_images.shape, test_images.shape, train_labels.shape, test_labels.shape

((5450, 64, 64, 3), (1363, 64, 64, 3), (5450, 6), (1363, 6))

In [9]:
len_text = 6
batch_size = 64
num_channels = 3
num_classes = 255
image_size = 72
latent_dim = 256

## Stage 1

In [10]:
def generate_c(x):
    mean = x[:, :128]
    log_sigma = x[:, 128:]
    stddev = K.exp(log_sigma)
    epsilon = K.random_normal(shape=K.constant((mean.shape[1],), dtype='int32'))
    c = stddev * epsilon + mean
    return c

In [11]:
def build_ca_model():
    """
    Get conditioning augmentation model.
    Takes an embedding of shape (1024,) and returns a tensor of shape (256,)
    """
    input_layer = Input(shape=(1024,))
    x = Dense(256)(input_layer)
    x = LeakyReLU(alpha=0.2)(x)
    model = Model(inputs=[input_layer], outputs=[x])
    return model

In [12]:
def build_embedding_compressor_model():
    """
    Build embedding compressor model
    """
    input_layer = Input(shape=(1024,))
    x = Dense(128)(input_layer)
    x = ReLU()(x)

    model = Model(inputs=[input_layer], outputs=[x])
    return model

In [13]:
def build_stage1_generator():
    """
    Builds a generator model used in Stage-I
    """
    input_layer = Input(shape=(1024,))
    x = Dense(256)(input_layer)
    mean_logsigma = LeakyReLU(alpha=0.2)(x)

    c = Lambda(generate_c)(mean_logsigma)

    input_layer2 = Input(shape=(100,))

    gen_input = Concatenate(axis=1)([c, input_layer2])

    x = Dense(128 * 8 * 4 * 4, use_bias=False)(gen_input)
    x = ReLU()(x)

    x = Reshape((4, 4, 128 * 8), input_shape=(128 * 8 * 4 * 4,))(x)

    x = UpSampling2D(size=(2, 2))(x)
    x = Conv2D(512, kernel_size=3, padding="same", strides=1, use_bias=False)(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)

    x = UpSampling2D(size=(2, 2))(x)
    x = Conv2D(256, kernel_size=3, padding="same", strides=1, use_bias=False)(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)

    x = UpSampling2D(size=(2, 2))(x)
    x = Conv2D(128, kernel_size=3, padding="same", strides=1, use_bias=False)(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)

    x = UpSampling2D(size=(2, 2))(x)
    x = Conv2D(64, kernel_size=3, padding="same", strides=1, use_bias=False)(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)

    x = Conv2D(3, kernel_size=3, padding="same", strides=1, use_bias=False)(x)
    x = Activation(activation='tanh')(x)

    stage1_gen = Model(inputs=[input_layer, input_layer2], outputs=[x, mean_logsigma])
    return stage1_gen

In [14]:
def build_stage1_discriminator():
    """
    Create a model which takes two inputs
    1. One from above network
    2. One from the embedding layer
    3. Concatenate along the axis dimension and feed it to the last module which produces final logits
    """
    input_layer = Input(shape=(64, 64, 3))

    x = Conv2D(64, (4, 4),
               padding='same', strides=2,
               input_shape=(64, 64, 3), use_bias=False)(input_layer)
    x = LeakyReLU(alpha=0.2)(x)

    x = Conv2D(128, (4, 4), padding='same', strides=2, use_bias=False)(x)
    x = BatchNormalization()(x)
    x = LeakyReLU(alpha=0.2)(x)

    x = Conv2D(256, (4, 4), padding='same', strides=2, use_bias=False)(x)
    x = BatchNormalization()(x)
    x = LeakyReLU(alpha=0.2)(x)

    x = Conv2D(512, (4, 4), padding='same', strides=2, use_bias=False)(x)
    x = BatchNormalization()(x)
    x = LeakyReLU(alpha=0.2)(x)

    input_layer2 = Input(shape=(4, 4, 128))

    merged_input = concatenate([x, input_layer2])

    x2 = Conv2D(64 * 8, kernel_size=1,
                padding="same", strides=1)(merged_input)
    x2 = BatchNormalization()(x2)
    x2 = LeakyReLU(alpha=0.2)(x2)
    x2 = Flatten()(x2)
    x2 = Dense(1)(x2)
    x2 = Activation('sigmoid')(x2)

    stage1_dis = Model(inputs=[input_layer, input_layer2], outputs=[x2])
    return stage1_dis

In [15]:
def build_adversarial_model(gen_model, dis_model):
    input_layer = Input(shape=(1024,))
    input_layer2 = Input(shape=(100,))
    input_layer3 = Input(shape=(4, 4, 128))

    x, mean_logsigma = gen_model([input_layer, input_layer2])

    dis_model.trainable = False
    valid = dis_model([x, input_layer3])

    model = Model(inputs=[input_layer, input_layer2, input_layer3], outputs=[valid, mean_logsigma])
    return model

### KL loss

In [16]:
def KL_loss(y_true, y_pred):
    mean = y_pred[:, :128]
    logsigma = y_pred[:, :128]
    loss = -logsigma + .5 * (-1 + K.exp(2. * logsigma) + K.square(mean))
    loss = K.mean(loss)
    return loss
    

def custom_generator_loss(y_true, y_pred):
    # Calculate binary cross entropy loss
    return K.binary_crossentropy(y_true, y_pred)

### save the generated image and logs

In [17]:
def save_rgb_img(img, path):
    """
    Save an rgb image
    """
    fig = plt.figure()
    ax = fig.add_subplot(1, 1, 1)
    ax.imshow(img)
    ax.axis("off")
    ax.set_title("Image")

    plt.savefig(path)
    plt.close()
    
def write_log(callback, name, loss, batch_no):
    """
    Write training summary to TensorBoard
    """
    callback.writer = tf.summary.create_file_writer(logdir="logs/".format(time.time()))
    with callback.writer.as_default():
        tf.summary.scalar(name, loss, step=batch_no)
        callback.writer.flush()


In [ ]:
if __name__ == '__main__':
    image_size = 64
    len_text = 6
    batch_size = 64
    z_dim = 100
    stage1_generator_lr = 0.0002
    stage1_discriminator_lr = 0.0002
    stage1_lr_decay_step = 600
    epochs = 1000
    condition_dim = 128
    
    word_embedding = keras.Sequential(
        [
            keras.layers.InputLayer(input_shape=(len_text,)),
            keras.layers.Embedding(vocab_size, 300, input_length=len_text),
            keras.layers.GRU(1024),
        ]
    )
    X_train = train_images
    X_test = test_images
    embeddings_train = word_embedding(train_labels)
    embeddings_test = word_embedding(test_labels)
    
     # Define optimizers
    dis_optimizer = Adam(lr=stage1_discriminator_lr, beta_1=0.5, beta_2=0.999)
    gen_optimizer = Adam(lr=stage1_generator_lr, beta_1=0.5, beta_2=0.999)
    
    ca_model = build_ca_model()
    ca_model.compile(loss="binary_crossentropy", optimizer="adam")

    stage1_dis = build_stage1_discriminator()
    stage1_dis.compile(loss='binary_crossentropy', optimizer=dis_optimizer)

    stage1_gen = build_stage1_generator()
    stage1_gen.compile(loss="mse", optimizer=gen_optimizer)

    embedding_compressor_model = build_embedding_compressor_model()
    embedding_compressor_model.compile(loss="binary_crossentropy", optimizer="adam")

    adversarial_model = build_adversarial_model(gen_model=stage1_gen, dis_model=stage1_dis)
    adversarial_model.compile(loss=['binary_crossentropy', KL_loss], loss_weights=[1, 2.0],
                              optimizer=gen_optimizer, metrics=None)

    tensorboard = TensorBoard()
    tensorboard.set_model(stage1_gen)
    tensorboard.set_model(stage1_dis)
    tensorboard.set_model(ca_model)
    tensorboard.set_model(embedding_compressor_model)

    # Generate an array containing real and fake values
    # Apply label smoothing as well
    real_labels = np.ones((batch_size, 1), dtype=float) * 0.9
    fake_labels = np.zeros((batch_size, 1), dtype=float) * 0.1
    
    for epoch in range(epochs):
        print("========================================")
        print("Epoch is:", epoch)
        print("Number of batches", int(X_train.shape[0] / batch_size))

        gen_losses = []
        dis_losses = []

        # Load data and train model
        number_of_batches = int(X_train.shape[0] / batch_size)
        for index in range(number_of_batches):
            print("Batch:{}".format(index+1))
            
            """
            Train the discriminator network
            """
            # Sample a batch of data
            z_noise = np.random.normal(0, 1, size=(batch_size, z_dim))
            image_batch = X_train[index * batch_size:(index + 1) * batch_size]
            embedding_batch = embeddings_train[index * batch_size:(index + 1) * batch_size]
            image_batch = (image_batch - 127.5) / 127.5

            # Generate fake images
            fake_images, _ = stage1_gen.predict([embedding_batch, z_noise], verbose=3)

            # Generate compressed embeddings
            compressed_embedding = embedding_compressor_model.predict_on_batch(embedding_batch)
            compressed_embedding = np.reshape(compressed_embedding, (-1, 1, 1, condition_dim))
            compressed_embedding = np.tile(compressed_embedding, (1, 4, 4, 1))

            dis_loss_real = stage1_dis.train_on_batch([image_batch, compressed_embedding],
                                                      np.reshape(real_labels, (batch_size, 1)))
            dis_loss_fake = stage1_dis.train_on_batch([fake_images, compressed_embedding],
                                                      np.reshape(fake_labels, (batch_size, 1)))
            dis_loss_wrong = stage1_dis.train_on_batch([image_batch[:(batch_size - 1)], compressed_embedding[1:]],
                                                       np.reshape(fake_labels[1:], (batch_size-1, 1)))

            d_loss = 0.5 * np.add(dis_loss_real, 0.5 * np.add(dis_loss_wrong, dis_loss_fake))

            print("d_loss_real:{}".format(dis_loss_real))
            print("d_loss_fake:{}".format(dis_loss_fake))
            print("d_loss_wrong:{}".format(dis_loss_wrong))
            print("d_loss:{}".format(d_loss))

            """
            Train the generator network 
            """
            g_loss = adversarial_model.train_on_batch([embedding_batch, z_noise, compressed_embedding],[K.ones((batch_size, 1)) * 0.9, K.ones((batch_size, 256)) * 0.9])
            print("g_loss:{}".format(g_loss))

            dis_losses.append(d_loss)
            gen_losses.append(g_loss)

        """
        Save losses to Tensorboard after each epoch
        """
        write_log(tensorboard, 'discriminator_loss', np.mean(dis_losses), epoch)
        write_log(tensorboard, 'generator_loss', np.mean(gen_losses[0]), epoch)
        
        # Generate and save images after every 2nd epoch
        if epoch % 2 == 0:
            # z_noise2 = np.random.uniform(-1, 1, size=(batch_size, z_dim))
            z_noise2 = np.random.normal(0, 1, size=(batch_size, z_dim))
            embedding_batch = embeddings_test[0:batch_size]
            fake_images, _ = stage1_gen.predict_on_batch([embedding_batch, z_noise2])

            # Save images
            for i, img in enumerate(fake_images[:10]):
                save_rgb_img(img, "results/gen_{}_{}.png".format(epoch, i))

    # Save models
    stage1_gen.save_weights("stage1_gen.h5")
    stage1_dis.save_weights("stage1_dis.h5")

2022-02-17 12:54:43.510461: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-02-17 12:54:43.510610: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Metal device set to: Apple M1 Max


/opt/homebrew/Caskroom/miniforge/base/envs/cp1/lib/python3.8/site-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


Epoch is: 0
Number of batches 85
Batch:1


2022-02-17 12:54:44.853914: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2022-02-17 12:54:44.907283: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2022-02-17 12:54:45.687924: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2022-02-17 12:54:45.968498: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2022-02-17 12:54:48.549275: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


...

g_loss:[0.7176743149757385, 0.7176707983016968, 1.7642287275521085e-06]
Epoch is: 81
Number of batches 85
Batch:1
d_loss_real:0.8109309673309326
d_loss_fake:3.314995774417184e-05
d_loss_wrong:0.6782684326171875
d_loss:0.5750408793091992
g_loss:[0.7199406623840332, 0.7199360132217407, 2.3380343918688595e-06]
Batch:2
d_loss_real:0.8336371183395386
d_loss_fake:2.0876879716524854e-05
d_loss_wrong:0.6377508044242859
d_loss:0.5762614794957699
g_loss:[0.7143465876579285, 0.7143428325653076, 1.8677520756682497e-06]
Batch:3
d_loss_real:0.821968674659729
d_loss_fake:1.2094905287085567e-05
d_loss_wrong:0.6248347759246826
d_loss:0.5671960550373569
g_loss:[0.7155542373657227, 0.7155516147613525, 1.3064025097264675e-06]
Batch:4
d_loss_real:0.8014804124832153
d_loss_fake:1.4124494555289857e-05
d_loss_wrong:0.6360201239585876
d_loss:0.5597487683548934
g_loss:[0.7055308818817139, 0.7055285573005676, 1.1480813100206433e-06]
Batch:5
d_loss_real:0.7882808446884155
d_loss_fake:1.1178233762620948e-05
d_loss

Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).


g_loss:[0.7094281315803528, 0.7094253301620483, 1.397630626343016e-06]


Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).


Epoch is: 83
Number of batches 85
Batch:1
d_loss_real:0.7960864901542664
d_loss_fake:2.2457172235590406e-05
d_loss_wrong:0.6666985154151917
d_loss:0.56472348822399
g_loss:[0.7137645483016968, 0.7137603759765625, 2.0919837879773695e-06]
Batch:2
d_loss_real:0.790608286857605
d_loss_fake:2.161021802749019e-05
d_loss_wrong:0.6536679863929749
d_loss:0.5587265425815531
g_loss:[0.7126988768577576, 0.7126953601837158, 1.7663127209743834e-06]
Batch:3
d_loss_real:0.79544597864151
d_loss_fake:1.4578957234334666e-05
d_loss_wrong:0.6016412973403931
d_loss:0.5481369583951619
g_loss:[0.7049985527992249, 0.7049957513809204, 1.4144891338219168e-06]
Batch:4
d_loss_real:0.8010509014129639
d_loss_fake:1.7494232452008873e-05
d_loss_wrong:0.6152574419975281
d_loss:0.554344184763977
g_loss:[0.7053980231285095, 0.7053956389427185, 1.1844053915410768e-06]
Batch:5
d_loss_real:0.7936645746231079
d_loss_fake:1.925914693856612e-05
d_loss_wrong:0.6304998397827148
d_loss:0.5544620620439673
g_loss:[0.703236997127533,